<br><font face="Times New Roman" size=5><div dir=ltr align=center>
<font color=blue size=8>
    Introduction to Machine Learning <br>
<font color=red size=5>
    Sharif University of Technology - Computer Engineering Department <br>
    Fall 2022<br> <br>
<font color=black size=6>
    Homework 2: Practical - Linear Regression
    </div>
<br><br>
<font size=4>
   **Name**: Lachin Naghashyar<br>
   **Student ID**: 98110179<br> <br>

<font face="Times New Roman" size=4><div dir=ltr>
# Problem 1: Linear Regression Model (40 + 30 optional points)
According to <a href="https://github.com/asharifiz/Introduction_to_Machine_Learning/blob/main/Jupyter_Notebooks/Chapter_02_Classical_Models/Linear%20regression.ipynb"><font face="Roboto">Linear Regression Notebook</font></a>, train a linear regression model on an arbitrary dataset. Explain your chosen dataset and split your data into train and test sets, then predict values for the test set using your trained model. Try to find the best hyperparameters for your model. (Using Lasso Regression, Ridge Regression or Elastic Net and comparing them will have extra optional points)
<br> Explain each step of your workflow.

## Preparing Data
I used the Real estate price prediction data set which was built for regression analysis and includes the date of purchase, house age, location, distance to nearest MRT station, and house price of unit area. We can take a look at our data below:

In [24]:
import pandas as pd

df = pd.read_csv('Real estate.csv')
df

,No,X1 transaction date,X2 house age,X3 distance to the nearest MRT station,X4 number of convenience stores,X5 latitude,X6 longitude,Y house price of unit area
0,1,2012.917,32.0,84.87882,10,24.98298,121.54024,37.9
1,2,2012.917,19.5,306.59470,9,24.98034,121.53951,42.2
2,3,2013.583,13.3,561.98450,5,24.98746,121.54391,47.3
3,4,2013.500,13.3,561.98450,5,24.98746,121.54391,54.8
4,5,2012.833,5.0,390.56840,5,24.97937,121.54245,43.1
...,...,...,...,...,...,...,...,...
409,410,2013.000,13.7,4082.01500,0,24.94155,121.50381,15.4
410,411,2012.667,5.6,90.45606,9,24.97433,121.54310,50.0
411,412,2013.250,18.8,390.96960,7,24.97923,121.53986,40.6
412,413,2013.000,8.1,104.81010,5,24.96674,121.54067,52.5


As you can see, it consists of different features for $X$ and also has the price of unit area as its final output ($Y$). In Regression, we are searching for relationships among variables. For example here, we are looking for a function that maps $X_i$ features to the price.
We can separate the data into $X$ and $Y$ arrays:

In [25]:
X = df.iloc[:, 1:7].to_numpy()
Y = df.iloc[:, 7:].to_numpy()

## Split data into Train and Test sets:
we use about 80 percent in the training and the rest in test set.

In [26]:
training_examples = int(X.shape[0] * 0.8)
X_train = X[:training_examples,:]
Y_train = Y[:training_examples,:]
X_test = X[training_examples:,:]
Y_test = Y[training_examples:,:]

We can also add an additional column to the $X$ array for the bias variable

In [27]:
import numpy as np

# X = np.append(np.ones((X.shape[0], 1)), X , axis=1)
# X

## Create a Model (using scikit-learn package)

In [28]:
from sklearn.linear_model import LinearRegression
model = LinearRegression()

Next we do the training using the fit function

In [29]:
model.fit(X_train, Y_train)

LinearRegression()

## Make predictions on the test set

In [30]:
from sklearn.metrics import mean_squared_error
predictions = model.predict(X_test)
mean_squared_error(Y_test, predictions)

59.40924577639657